In [1]:
from torchvision import transforms
import torchvision
import torch
from PIL import Image
import numpy as np
import os

In [2]:
FRAME_DIR = "training_set/Frames"
FEATURES_DIR = "training_set/Features"

In [3]:
resnet152 = torchvision.models.resnet152(pretrained=True)

In [4]:
# Since we want the feature vector, we replace the final fc layer of the model with an identity layer
class Identity(torch.nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

resnet152.fc = Identity()
resnet152.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
# this probably could be sped up using CUDA but eh whatever it's only a few minutes
def extract_frame_features(model, model_name, preprocess):
    if not os.path.exists(f"{FEATURES_DIR}/{model_name}"):
        os.mkdir(f"{FEATURES_DIR}/{model_name}")

    for frame_file in os.listdir(FRAME_DIR):
        feature_path = f"{FEATURES_DIR}/{model_name}/{frame_file[:-4]}.csv"
        if not os.path.exists(feature_path):
            input_image = Image.open(f"{FRAME_DIR}/{frame_file}")
            input_tensor = preprocess(input_image)
            input_batch = input_tensor.unsqueeze(0)
            with torch.no_grad():
                output = model(input_batch)[0]
            np.savetxt(feature_path, output.numpy(), delimiter=',')

In [6]:
# Required preprocessing for resnet
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [7]:
extract_frame_features(resnet152, "ResNet152", preprocess)